In [69]:
import numpy as np
import torch
## 1차원인 경우를 생각해서 해보자
c = torch.randn(1,3)
d = torch.randn(1,3)
print(c.shape, d.shape)
# 분모 
vec_c = np.linalg.norm(c)
vec_d = np.linalg.norm(d)

# 분자
son = np.dot(c,d.T)

# 계산
print(son / (vec_c * vec_d))

## pytorch cos
cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6) # dimention 1은 열의 값을 가지고 내적하는 것
print(cos(c,d))

torch.Size([1, 3]) torch.Size([1, 3])
[[0.5090391]]
tensor([0.5090])


In [11]:
# 2차원인 경우
a = torch.randn(2,3) # 2 by 3 행렬
b = torch.randn(2,3)
print(a)
print(b)

tensor([[-1.4957, -1.9285, -0.2320],
        [-0.3472, -1.1279,  0.3151]])
tensor([[-0.9709,  0.4931, -0.2877],
        [ 1.9940,  1.8288, -0.5981]])


In [96]:
# 분모 = 두 벡터 크기의 곱
vector_a= np.linalg.norm(a[0])
vector_b = np.linalg.norm(b[0])
vector_x = np.linalg.norm(a[1])
vector_y = np.linalg.norm(b[1])
print(vector_a, vector_b)
print(vector_x ,vector_y)
print('=' * 10)

2.451584 1.1263523
1.2214428 2.7709858


In [97]:
# 분자 = 두 벡터의 내적
numerator_ab = np.dot(a[0],b[0].T)
numerator_xy = np.dot(a[1],b[1].T)
print(numerator_ab, numerator_xy)

0.5679307 -2.9434323


In [102]:
# 코사인 유사도 계산
print(f"Cosine Sim of First Row: {numerator_ab / (vector_a * vector_b):.4f}")
print(f"Cosine Sim of Second Row: {numerator_xy / (vector_x * vector_y):.4f}")

# 확인하기 위한 torch.nn.CosineSimilarity
cos = torch.nn.CosineSimilarity(dim = 1, eps = 1e-6)
print(cos(a,b))

Cosine Sim of First Row: 0.2057
Cosine Sim of Second Row: -0.8697
tensor([ 0.2057, -0.8697])


---

SimCLR에서 사용하는 Cosine Similarity 계산해보기

In [12]:
## 그렇다면 SSL에서 사용한 Cosinesim dim =2는 어떻게 계산이 될까
# 해당 코드에서 data의 shape은 (32,1,28,28) 흑백이여서 channel수가 1이다
import torch
import numpy as np
# SSL 코드에서는 pair로 들어갔기 때문이다.
a0 = torch.randn(32,1,28,28) # original data
a1 = torch.randn(32,1,28,28) # augmentation data
print(a0.shape)
print(a1.shape)

torch.Size([32, 1, 28, 28])
torch.Size([32, 1, 28, 28])


In [17]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5, stride=1)
        self.fc = nn.Linear(4 * 4 * 20, 100)

    def forward(self, x):
        x = F.relu(self.conv1(x)) # (batch, 1, 28, 28) -> (batch, 10, 24, 24)

        x = F.max_pool2d(x, kernel_size=2, stride=2) # (batch, 10, 24, 24) -> (batch, 10, 12, 12)

        x = F.relu(self.conv2(x)) # (batch, 10, 12, 12) -> (batch, 20, 8, 8)

        x = F.max_pool2d(x, kernel_size=2, stride=2) # (batch, 20, 8, 8) -> (batch, 20, 4, 4)

        x = x.view(-1, 4 * 4 * 20) # (batch, 20, 4, 4) -> (batch, 320)

        x = F.relu(self.fc(x)) # (batch, 320) -> (batch, 100)
        return x # (batch, 100)
model = CNN()

# 위 모델에 a0와 a1의 데이터가 들어가서 embedding을 만들어 냄
positive = model(a0)
negative = model(a1)

z = torch.cat((positive, negative), dim=0) 
print(z.shape)

torch.Size([64, 100])


In [19]:
# 코사인 유사도를 계산할 때 unsqueeze(0)과 1을 넣어서 계산을 해주게 된다. 
print(z.unsqueeze(0).shape)
print(z.unsqueeze(1).shape)

torch.Size([1, 64, 100])
torch.Size([64, 1, 100])


In [23]:
a = z.unsqueeze(0)
b = z.unsqueeze(1)
cos = torch.nn.CosineSimilarity(dim = 2)
cos(a,b).shape

torch.Size([64, 64])

In [35]:
# dim =2를 해주었다는 이야기는 C (2차원)의 데이터를 가지고 코사인 유사도를 측정하겠다는 것이니
cos_list = []
for x in a:
    x = x.detach().numpy()
    vector_a = np.linalg.norm(x) # 벡터의 크기
    for y in b:
        y= y.detach().numpy()
        vector_b = np.linalg.norm(y) # 벡터의 크기

        numerator = np.dot(x,y.T) # 분모

        ## cosine 유사도 계산
        cosnie_sim = numerator / (vector_a * vector_b)
        cos_list.append(cosnie_sim)

# print(cos_list)
print(len(cos_list))
print(len(cos_list[0]))



64
64


In [38]:
cos_list[0]

array([[0.12915708],
       [0.12252176],
       [0.1328124 ],
       [0.1169735 ],
       [0.12281352],
       [0.11899776],
       [0.11801495],
       [0.11059094],
       [0.11588106],
       [0.11815374],
       [0.1134853 ],
       [0.11922606],
       [0.11015929],
       [0.12637682],
       [0.11286192],
       [0.11396182],
       [0.11444164],
       [0.11045194],
       [0.11191768],
       [0.11539806],
       [0.12780277],
       [0.12572479],
       [0.12006505],
       [0.10807674],
       [0.12459756],
       [0.12320335],
       [0.1147337 ],
       [0.1153216 ],
       [0.12477203],
       [0.10225712],
       [0.12090342],
       [0.1186257 ],
       [0.10002552],
       [0.11033254],
       [0.11839703],
       [0.1131257 ],
       [0.11126592],
       [0.11267783],
       [0.111331  ],
       [0.1175622 ],
       [0.12329023],
       [0.10934812],
       [0.11716717],
       [0.12531376],
       [0.11777568],
       [0.11805354],
       [0.12869275],
       [0.116

In [37]:
print(cos(a,b))

## shape은 같으나 값이 다르다

tensor([[1.0000, 0.9359, 0.9334,  ..., 0.9379, 0.9366, 0.9360],
        [0.9359, 1.0000, 0.9388,  ..., 0.9290, 0.9311, 0.9424],
        [0.9334, 0.9388, 1.0000,  ..., 0.9182, 0.9506, 0.9465],
        ...,
        [0.9379, 0.9290, 0.9182,  ..., 1.0000, 0.9331, 0.9347],
        [0.9366, 0.9311, 0.9506,  ..., 0.9331, 1.0000, 0.9431],
        [0.9360, 0.9424, 0.9465,  ..., 0.9347, 0.9431, 1.0000]],
       grad_fn=<SumBackward1>)
